In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920x1080")

options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

categorys = ["001001001", "001001002", "001001008", "001001010", "001001003", "001001011", "001001007" ]

for category in categorys:
    url = f"https://display.wconcept.co.kr/category/women/{category}"
    driver.get(url)
    time.sleep(3)

    print(f"\n\n========== {category} 카테고리 여성의류==========")

    idx = 0
    while idx < 30:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "button.area-img")
            if idx >= len(buttons):
                break

            driver.execute_script("arguments[0].click();", buttons[idx])
            time.sleep(3)

            print(f"\n----- {category} {idx+1}번째 의류 -----")

            try:
                title = driver.find_element(By.CSS_SELECTOR, "h2.brand").text.strip()
            except NoSuchElementException:
                title = "상품명 없음"

            try:
                rate = driver.find_element(By.CSS_SELECTOR, "p.grade_num").text.strip()
            except NoSuchElementException:
                rate = "0"

            try:
                review_num = driver.find_element(By.CSS_SELECTOR, "#reviewCnt1").text.strip()
            except NoSuchElementException:
                review_num = "0"

            # 리뷰 (최대 3개)
            reviews = []
            try:
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(0.5)
                review_button = driver.find_element(By.CSS_SELECTOR, "#detail > li:nth-child(2)")
                review_button.click()
                time.sleep(3)

                review_elements = driver.find_elements(By.CSS_SELECTOR, "p.pdt_review_text")
                reviews = [r.text.strip() for r in review_elements[:3]]

                if not reviews:
                    reviews = ["댓글 정보 없음"]

            except NoSuchElementException:
                reviews = ["댓글 정보 없음"]

            print("상품명:", title)
            print("평점:", rate)
            print("리뷰수:", review_num)
            for i, review in enumerate(reviews, start=1):
                print(f"리뷰{i}:", review)

            # save_to_db(category, title, rate, review_num)

            driver.back()
            time.sleep(1)
            driver.back()
            time.sleep(2)
            driver.execute_script("window.scrollBy(0, 200);")
            time.sleep(1)

            idx += 1

        except StaleElementReferenceException:
            print(f"{idx+1}번째 요소 stale 참조 발생, 재시도")
            time.sleep(1)
            continue

driver.quit()



========== 001001001 카테고리 여성의류==========

----- 001001001 1번째 의류 -----
상품명: Dunst for WOMEN
평점: 5
리뷰수: 8
리뷰1: 브라운 색상으로 사이즈 진짜 고민 많이 했는데
키가 커서 m했는데 아주 찰떡 !
평소에도 아우터는 한치수나 두치수 업해서
입기때문에 사이즈 아주 굿 좋아요
리뷰2: 던스트 신상나와서 바로 구매했어요!
아우터는 조금 크게나오는듯해서 XS로 주문했는데 잘 주문한것같아요!! S했으면 어정쩡하게 커서 어색했을것같아요!! 입어봤는데 박시핏에 길이도 넉넉해서 예뻐요!!
리뷰3: 예뻐요..! S랑 고민했는데 XS 하길 너무 잘했어요 ! 색도 너무 예쁘고 두루두루 어울릴 거 같아요!

----- 001001001 2번째 의류 -----
상품명: IvanaHelsinki단독
평점: 4.9
리뷰수: 610
리뷰1: 디자인이랑 컬러는 사진이랑 똑같아요 맘에듬!
사이즈는 평소에 s사이즈 자켓입는데 작은듯하면서 따악 맞는느낌이라 요즘 살이좀 쪘나싶은 ㅎㅎ 옷이이쁘니까 그냥 조금만 빼서 입으면 될듯용 하나큰거샀음 좀 애매했을거같아요!
리뷰2: 상견례용으로 구입했는데 옷은 예뻐요
근데 다들 어깨뽕 어떻게 하셨나요? 잘 맞으시나요..?
리뷰3: 가슴이 좀 큰 편이라 플러스 사이즈 구매했는데 불편하지 않고 좋아요!! 스커트와 함께 입으니 정말 예뻐요~


KeyboardInterrupt: 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import time
import mysql.connector


conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="dnwhdgus9#"
)
cursor = conn.cursor()

# 데이터베이스 생성
cursor.execute("CREATE DATABASE IF NOT EXISTS wconcept CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
conn.database = "wconcept"

# 테이블 생성 (리뷰 최대 3개 포함)
cursor.execute("""
CREATE TABLE IF NOT EXISTS performances1 (
    id INT AUTO_INCREMENT PRIMARY KEY,
    category VARCHAR(50),
    title VARCHAR(100),
    rate VARCHAR(100),
    review_num VARCHAR(100),
    review1 TEXT,
    review2 TEXT,
    review3 TEXT,
    UNIQUE KEY unique_performance (title, rate, review_num)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
""")
conn.commit()

# ==============================
# 2. DB 저장 함수
# ==============================
def save_to_db(category, title, rate, review_num, reviews):
    # 리뷰 3개 맞추기 (없으면 "댓글 정보 없음")
    reviews += ["댓글 정보 없음"] * (3 - len(reviews))
    sql = """
    INSERT IGNORE INTO performances1
    (category, title, rate, review_num, review1, review2, review3) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(sql, (category, title, rate, review_num, reviews[0], reviews[1], reviews[2]))
    conn.commit()

# ==============================
# 3. Selenium 설정
# ==============================
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
driver = webdriver.Chrome(service=service, options=options)

# ==============================
# 4. 크롤링
# ==============================
categorys = ["001001001", "001001002", "001001008", "001001010", "001001003", "001001011", "001001007"]

for category in categorys:
    url = f"https://display.wconcept.co.kr/category/women/{category}"
    driver.get(url)
    time.sleep(2)
    print(f"\n\n========== {category} 카테고리 여성의류 ==========")
    
    idx = 0
    while idx < 30:
        try:
            buttons = driver.find_elements(By.CSS_SELECTOR, "button.area-img")
            if idx >= len(buttons):
                break
            # 상세페이지에 들어가는 코드
            driver.execute_script("arguments[0].click();", buttons[idx])
            time.sleep(2)
            print(f"\n----- {category} {idx+1}번째 의류 -----")
            
            # 상품명
            try:
                title = driver.find_element(By.CSS_SELECTOR, "h2.brand").text.strip()
            except NoSuchElementException:
                title = "상품명 없음"
            
            # 평점
            try:
                rate = driver.find_element(By.CSS_SELECTOR, "p.grade_num").text.strip()
            except NoSuchElementException:
                rate = "0"
            
            # 리뷰수
            try:
                review_num = driver.find_element(By.CSS_SELECTOR, "#reviewCnt1").text.strip()
            except NoSuchElementException:
                review_num = "0"
            
            # 리뷰 최대 3개
            reviews = []
            try:
                driver.execute_script("window.scrollBy(0, 300);")
                time.sleep(0.5)
                review_button = driver.find_element(By.CSS_SELECTOR, "#detail > li:nth-child(2)")
                review_button.click()
                time.sleep(1.5)
                
                review_elements = driver.find_elements(By.CSS_SELECTOR, "p.pdt_review_text")
                reviews = [r.text.strip() for r in review_elements[:3]]
                if not reviews:
                    reviews = ["댓글 정보 없음"]
            except NoSuchElementException:
                reviews = ["댓글 정보 없음"]
            
            # 출력
            print("상품명:", title)
            print("평점:", rate)
            print("리뷰수:", review_num)
            for i, review in enumerate(reviews, start=1):
                print(f"리뷰{i}:", review)
            
            # DB 저장
            save_to_db(category, title, rate, review_num, reviews)
            
            # 뒤로가기
            driver.back()
            time.sleep(0.5)
            driver.back()
            time.sleep(1)
            driver.execute_script("window.scrollBy(0, 100);")
            time.sleep(1)
            
            idx += 1
        except StaleElementReferenceException:
            print(f"{idx+1}번째 요소 stale 참조 발생, 재시도")
            time.sleep(1)
            continue

# ==============================
# 5. 종료
# ==============================
driver.quit()
cursor.close()
conn.close()



========== 001001001 카테고리 여성의류 ==========

----- 001001001 1번째 의류 -----
상품명: Dunst for WOMEN
평점: 5
리뷰수: 8
리뷰1: 브라운 색상으로 사이즈 진짜 고민 많이 했는데
키가 커서 m했는데 아주 찰떡 !
평소에도 아우터는 한치수나 두치수 업해서
입기때문에 사이즈 아주 굿 좋아요
리뷰2: 던스트 신상나와서 바로 구매했어요!
아우터는 조금 크게나오는듯해서 XS로 주문했는데 잘 주문한것같아요!! S했으면 어정쩡하게 커서 어색했을것같아요!! 입어봤는데 박시핏에 길이도 넉넉해서 예뻐요!!
리뷰3: 예뻐요..! S랑 고민했는데 XS 하길 너무 잘했어요 ! 색도 너무 예쁘고 두루두루 어울릴 거 같아요!

----- 001001001 2번째 의류 -----
상품명: IvanaHelsinki단독
평점: 4.9
리뷰수: 610
리뷰1: 디자인이랑 컬러는 사진이랑 똑같아요 맘에듬!
사이즈는 평소에 s사이즈 자켓입는데 작은듯하면서 따악 맞는느낌이라 요즘 살이좀 쪘나싶은 ㅎㅎ 옷이이쁘니까 그냥 조금만 빼서 입으면 될듯용 하나큰거샀음 좀 애매했을거같아요!
리뷰2: 상견례용으로 구입했는데 옷은 예뻐요
근데 다들 어깨뽕 어떻게 하셨나요? 잘 맞으시나요..?
리뷰3: 가슴이 좀 큰 편이라 플러스 사이즈 구매했는데 불편하지 않고 좋아요!! 스커트와 함께 입으니 정말 예뻐요~

----- 001001001 3번째 의류 -----
상품명: MangoManyPlease
평점: 0
리뷰수: 0
리뷰1: 댓글 정보 없음

----- 001001001 4번째 의류 -----
상품명: ADIDAS
평점: 4.9
리뷰수: 46
리뷰1: 역시 아디다스네요.
기본컬러 트랙탑이라 정말 1도 고민 안하고 구매했어요.
역시나 핏도 길이도 너무 맘에 들고 화이트 롱스커트에 잠깐 매치해봤는데도 아묻따 예쁘네요.
리뷰2: 라지 사이즈 적당하고 딱 좋아요 키165에 77사이즈 날씬해보이네요
리뷰3: 얇은 너무박시하지 않은 딱 원하던 ^^ 너무잘